<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [1]:
import torch
import torchvision
import psutil
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision import transforms
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class CigaretteDataset(Dataset):
    def __init__(self, coco_annotation_file, image_dir, transform=None):
        self.coco = COCO(coco_annotation_file)
        self.image_dir = image_dir
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        cat_ids = self.coco.getCatIds(catNms=['cigarette', 'smoker', 'non_smoker'])
        if not cat_ids:
            raise ValueError("No 'cigarette' category found in COCO file.")
        self.image_ids = list(set(self.coco.getImgIds(catIds=cat_ids)))

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        image = Image.open(f"{self.image_dir}/{img_info['file_name']}").convert("RGB")

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        annotations = self.coco.loadAnns(ann_ids)
        boxes, labels = [], []

        for ann in annotations:
            if ann['category_id'] in self.coco.getCatIds(catNms=['cigarette']):
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(1)  # Class 1 for 'cigarette'

        if len(boxes) == 0:
            return None

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        image = self.transform(image)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }

        return image, target

In [4]:
def collate_fn(batch):
    return tuple(zip(*[b for b in batch if b is not None]))

In [5]:
def check_system_usage():
    print(f"CPU Usage: {psutil.cpu_percent()}%")
    print(f"RAM Usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

AP (Average Precision) a vari livelli di IoU,
AR (Average Recall) per varie quantità di detections,
mAP (mean Average Precision).

In [6]:
def evaluate_model(model, dataset, device):
    print("\n=== Starting Validation ===")
    model.eval()
    coco_dt = []
    coco_gt = dataset.coco

    # Processo di validazione con progress bar
    with torch.no_grad():
        for idx in tqdm(range(len(dataset)), desc="Validating", ncols=100):
            image, target = dataset[idx]
            image = image.to(device)
            prediction = model([image])

            for box, score, label in zip(prediction[0]['boxes'], prediction[0]['scores'], prediction[0]['labels']):
                x, y, w, h = box.tolist()
                coco_dt.append({
                    'image_id': target['image_id'].item(),
                    'category_id': label.item(),
                    'bbox': [x, y, w-x, h-y],  # Converti in formato COCO [x,y,width,height]
                    'score': score.item()
                })

    print(f"\nProcessed {len(coco_dt)} detections")

    if len(coco_dt) == 0:
        print("No detections found during validation!")
        return

    try:
        coco_dt = coco_gt.loadRes(coco_dt)
        coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')

        # Valuta per ogni categoria
        categories = dataset.coco.loadCats(dataset.coco.getCatIds())
        for cat in categories:
            coco_eval.params.catIds = [cat['id']]
            coco_eval.evaluate()
            coco_eval.accumulate()
            print(f"\nResults for {cat['name']}:")
            coco_eval.summarize()

        # Valutazione complessiva
        coco_eval.params.catIds = dataset.coco.getCatIds()
        coco_eval.evaluate()
        coco_eval.accumulate()
        print("\nOverall Results:")
        coco_eval.summarize()
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")

    print("\n=== Validation Complete ===\n")

In [7]:
def train_model(dataset, num_epochs=10, val_dataset=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize model and move to device
    model = fasterrcnn_resnet50_fpn(weights='DEFAULT')
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=4)
    model = model.to(device)

    data_loader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    # Use newer autocast and GradScaler API
    scaler = torch.amp.GradScaler('cuda') if torch.cuda.is_available() else None
    epoch_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        start_time = time.time()
        print(f"Starting Epoch {epoch+1}/{num_epochs}")

        for batch_idx, (images, targets) in enumerate(tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100, unit="batch")):
            # Move images to device
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()

            if scaler is not None:
                with torch.amp.autocast('cuda'):
                    loss_dict = model(images, targets)
                    loss = sum(loss for loss in loss_dict.values())

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss_dict = model(images, targets)
                loss = sum(loss for loss in loss_dict.values())
                loss.backward()
                optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(data_loader)
        epoch_losses.append(avg_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Time: {time.time() - start_time:.2f}s")
        check_system_usage()

        if val_dataset:
            evaluate_model(model, val_dataset, device)
        else:
            print("No validation dataset provided - skipping validation")

    torch.save(model.state_dict(), "fasterrcnn_cigarette.pth")
    return model

In [8]:
if __name__ == "__main__":
    #psutil.Process().nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    train_image_dir = '/content/drive/MyDrive/Photo/train'
    train_coco_annotation_file = '/content/drive/MyDrive/Photo/train/_annotations.coco.json'

    valid_image_dir = '/content/drive/MyDrive/Photo/valid'
    valid_coco_annotation_file = '/content/drive/MyDrive/Photo/valid/_annotations.coco.json'

    dataset = CigaretteDataset(train_coco_annotation_file, train_image_dir)
    val_dataset = CigaretteDataset(valid_coco_annotation_file, valid_image_dir)

    model = train_model(dataset, num_epochs=10, val_dataset= val_dataset)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Using device: cuda
Starting Epoch 1/10


Epoch 1/10: 100%|████████████████████████████████████████████████| 92/92 [01:55<00:00,  1.25s/batch]


Epoch 1/10, Loss: 0.4247, Time: 115.24s
CPU Usage: 85.7%
RAM Usage: 33.6%
GPU Memory Allocated: 0.80 GB
GPU Memory Reserved: 11.17 GB

=== Starting Validation ===


Validating: 100%|█████████████████████████████████████████████████| 142/142 [03:02<00:00,  1.29s/it]



Processed 1658 detections
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).

Results for smoking-6o0y-mGeT:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000


...

Epoch 4/10, Loss: 0.1722, Time: 111.79s
CPU Usage: 88.4%
RAM Usage: 33.8%
GPU Memory Allocated: 0.80 GB
GPU Memory Reserved: 11.17 GB

=== Starting Validation ===


Validating: 100%|█████████████████████████████████████████████████| 142/142 [00:30<00:00,  4.58it/s]



Processed 654 detections
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).

Results for smoking-6o0y-mGeT:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 

Epoch 5/10:  40%|███████████████████▎                            | 37/92 [00:46<01:08,  1.25s/batch]


KeyboardInterrupt: 